<a href="https://colab.research.google.com/github/EastHuni/lg-aimers/blob/main/%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89_%EC%84%BC%EC%84%9C%EC%9D%98_%EC%95%88%ED%85%8C%EB%82%98_%EC%84%B1%EB%8A%A5_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# import os

# zip_path = '/content/drive/MyDrive/LG AImers/LG Aimers 1기_data.zip'
# extract_path = '/content/drive/MyDrive/LG Almers/LG Aimers 1기_datasets'

# os.makedirs(extract_path, exist_ok=True)

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print("압축 해제 완료!")

In [ ]:
def lg_nrmse(gt, preds):
    all_nrmse = []
    for idx in range(14):  # Y_01 ~ Y_14
        mse = metrics.mean_squared_error(gt[:, idx], preds[:, idx])  # squared=False 제거
        rmse = np.sqrt(mse)  # 직접 제곱근 계산
        nrmse = rmse / np.mean(np.abs(gt[:, idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:])
    return score

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/LG Almers/LG Aimers 1기_datasets/train.csv")
test = pd.read_csv("/content/drive/MyDrive/LG Almers/LG Aimers 1기_datasets/test.csv")

In [ ]:
print(train.shape, test.shape)

(39607, 71) (39608, 57)


## 변수제거 부분 추가  

In [ ]:
drop_cols = ['X_02', 'X_04', 'X_23', 'X_47', 'X_48']
train = train.drop(drop_cols, axis = 1)
train.head()

,ID,X_01,X_03,X_05,X_06,X_07,X_08,X_09,X_10,X_11,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,67.47,101.892,74.983,29.45,62.38,245.71,0.0,0.0,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,65.17,101.944,72.943,28.73,61.23,233.61,0.0,0.0,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,64.07,103.153,72.943,28.81,105.77,272.20,0.0,0.0,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,67.57,101.971,77.022,28.92,115.21,255.36,0.0,0.0,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,63.57,101.981,70.904,29.68,103.38,241.46,0.0,0.0,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np


X = train.filter(regex='X') # Input : X Featrue
y = train.filter(regex='Y') # Output : Y Feature
print(X.shape, y.shape)

(39607, 51) (39607, 14)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

# 1. 데이터 로드
train = pd.read_csv("/content/drive/MyDrive/LG Almers/LG Aimers 1기_datasets/train.csv")
test = pd.read_csv("/content/drive/MyDrive/LG Almers/LG Aimers 1기_datasets/test.csv")

# 2. 입력/출력 나누기
X = train.filter(regex='^X')
y = train.filter(regex='^Y')
print(f"X shape: {X.shape}, y shape: {y.shape}")

# 3. 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. LG NRMSE 평가 함수
def lg_nrmse(gt, preds):
    all_nrmse = []
    for idx in range(14):  # Y_01 ~ Y_14
        mse = metrics.mean_squared_error(gt[:, idx], preds[:, idx])  # squared=False 제거
        rmse = np.sqrt(mse)  # 직접 제곱근 계산
        nrmse = rmse / np.mean(np.abs(gt[:, idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:])
    return score


# 5. RandomForest (스케일링 없음)
rf = RandomForestRegressor(n_estimators=20, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_val)
rf_score = lg_nrmse(y_val.values, rf_preds)

# 6. LinearRegression (스케일링 적용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_preds = lr.predict(X_val_scaled)
lr_score = lg_nrmse(y_val.values, lr_preds)

# 7. 결과 출력
results = pd.DataFrame({
    'Model': ['RandomForest (20 trees)', 'LinearRegression'],
    'LG_NRMSE': [rf_score, lr_score]
})
print("\n[모델 성능 비교]")
print(results)


X shape: (39607, 56), y shape: (39607, 14)

[모델 성능 비교]
                     Model  LG_NRMSE
0  RandomForest (20 trees)  1.992262
1         LinearRegression  1.971303


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
# 5. RandomForest (스케일링 없음)
rf = RandomForestRegressor(n_estimators=20, max_depth = 9, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_val)
rf_score = lg_nrmse(y_val.values, rf_preds)

# 6. LinearRegression (스케일링 적용)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_preds = lr.predict(X_val_scaled)
lr_score = lg_nrmse(y_val.values, lr_preds)

results = pd.DataFrame({
    'Model': ['RandomForest (20 trees)', 'LinearRegression'],
    'LG_NRMSE': [rf_score, lr_score]
})
print("\n[모델 성능 비교]")
print(results)


[모델 성능 비교]
                     Model  LG_NRMSE
0  RandomForest (20 trees)  1.968396
1         LinearRegression  1.971303


In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_preds = lr.predict(X_val_scaled)
lr_score = lg_nrmse(y_val.values, lr_preds)
print(f"[LinearRegression (MinMax)] LG_NRMSE: {lr_score:.4f}")

[LinearRegression (MinMax)] LG_NRMSE: 1.9713


In [ ]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgb_base = XGBRegressor(n_estimators=100, max_depth=5, random_state=42)
xgb = MultiOutputRegressor(xgb_base)

xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_val)
score = lg_nrmse(y_val.values, xgb_preds)
print(f"[XGBoost (max_depth=5)] LG_NRMSE: {score:.4f}")

[XGBoost (max_depth=5)] LG_NRMSE: 1.9764


In [ ]:
rf = RandomForestRegressor(n_estimators=30, max_depth = 9, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_val)
rf_score = lg_nrmse(y_val.values, rf_preds)
print(f"[RandomForest (max_depth=9)] LG_NRMSE: {rf_score:.4f}")

[RandomForest (max_depth=9)] LG_NRMSE: 1.9613


In [ ]:
# 1. RandomForest -- baseline(n_estimators=20, random_state=42) : 1.992262
# 2. LinearRegreession(Standard Scaler) : 1.971303
# 3. RandomForest(baseline + maxdepth = 7) : 1.968396 # 230등
# 4. LinearRegreession(RobustScaler), LinearRegression (MinMax) : 1.971303 // 변화없음
# 5. XGboost(n_estimators=100, max_depth=5, random_state=42) : 1.9764
# ----------------------------------------------------------------------------
# 위에서 젤 높은거 RandomForest 로 튜닝해보겟음
# 1. RandomForest(baseline + maxdepth = 9) : 1.9627 # 134등
# 2. RandomForestRegressor(n_estimators=30, max_depth = 9, random_state=42) : 1.9613 #110등

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.8 MB/s eta 0:00:00


In [ ]:
# import optuna
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.multioutput import MultiOutputRegressor

# def objective(trial):
#     # 하이퍼파라미터 범위 설정

#     max_depth = trial.suggest_int('max_depth', 3, 15)
#     n_estimators = trial.suggest_int('n_estimators', 10, 100)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

#     rf = RandomForestRegressor(
#         max_depth=max_depth,
#         n_estimators=n_estimators,
#         min_samples_leaf=min_samples_leaf,
#         random_state=42
#     )

#     model = MultiOutputRegressor(rf)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_val)

#     score = lg_nrmse(y_val.values, preds)
#     return score  # optuna는 낮은 값이 더 좋은 것으로 간주함

In [ ]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor


optuna.logging.set_verbosity(optuna.logging.INFO)


storage = "sqlite:///rf_optuna(del).db"

study = optuna.create_study(
    direction="minimize",
    study_name="rf_tuning",
    storage=storage,
    load_if_exists=True
)

def objective(trial):
    max_depth = trial.suggest_int('max_depth', 3, 12)
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestRegressor(
        max_depth=max_depth,
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )
    model = MultiOutputRegressor(model)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = lg_nrmse(y_val.values, preds)

    print(f"[Trial {trial.number}] score={score:.4f} | depth={max_depth}, n_estimators={n_estimators}, min_leaf={min_samples_leaf}")
    return score

study.optimize(objective, n_trials=50)

[I 2025-07-11 13:36:39,082] Using an existing study with name 'rf_tuning' instead of creating a new one.
[I 2025-07-11 13:44:01,518] Trial 1 finished with value: 1.9428029772714253 and parameters: {'max_depth': 10, 'n_estimators': 72, 'min_samples_leaf': 9}. Best is trial 1 with value: 1.9428029772714253.


[Trial 1] score=1.9428 | depth=10, n_estimators=72, min_leaf=9


[I 2025-07-11 13:46:39,178] Trial 2 finished with value: 1.9719074551872937 and parameters: {'max_depth': 3, 'n_estimators': 87, 'min_samples_leaf': 10}. Best is trial 1 with value: 1.9428029772714253.


[Trial 2] score=1.9719 | depth=3, n_estimators=87, min_leaf=10


[I 2025-07-11 13:51:53,621] Trial 3 finished with value: 1.9464775617992447 and parameters: {'max_depth': 8, 'n_estimators': 63, 'min_samples_leaf': 8}. Best is trial 1 with value: 1.9428029772714253.


...

[Trial 12] score=1.9403 | depth=12, n_estimators=99, min_leaf=6


[I 2025-07-11 14:39:43,911] Trial 13 finished with value: 1.9402275876442183 and parameters: {'max_depth': 12, 'n_estimators': 98, 'min_samples_leaf': 6}. Best is trial 11 with value: 1.9402275876442183.


[Trial 13] score=1.9402 | depth=12, n_estimators=98, min_leaf=6


[I 2025-07-11 14:51:13,104] Trial 14 finished with value: 1.9403024578287804 and parameters: {'max_depth': 12, 'n_estimators': 100, 'min_samples_leaf': 5}. Best is trial 11 with value: 1.9402275876442183.


[Trial 14] score=1.9403 | depth=12, n_estimators=100, min_leaf=5


In [ ]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=30)

# print("Best Params:", study.best_params)
# print("Best Score:", study.best_value)


[I 2025-07-10 16:06:58,280] A new study created in memory with name: no-name-28e27f39-e64f-480d-adc8-8439f2ffb57b
[I 2025-07-10 16:15:07,290] Trial 0 finished with value: 1.9489668821607211 and parameters: {'max_depth': 7, 'n_estimators': 88, 'min_samples_leaf': 6}. Best is trial 0 with value: 1.9489668821607211.
[I 2025-07-10 16:28:37,203] Trial 1 finished with value: 1.9421799641969546 and parameters: {'max_depth': 11, 'n_estimators': 94, 'min_samples_leaf': 4}. Best is trial 1 with value: 1.9421799641969546.
[I 2025-07-10 16:32:52,935] Trial 2 finished with value: 1.9520586477854869 and parameters: {'max_depth': 9, 'n_estimators': 36, 'min_samples_leaf': 1}. Best is trial 1 with value: 1.9421799641969546.
[W 2025-07-10 16:39:55,610] Trial 3 failed with parameters: {'max_depth': 10, 'n_estimators': 59, 'min_samples_leaf': 7} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py

KeyboardInterrupt: 

In [ ]:
# Trial 0 finished with value: 1.9489668821607211 and parameters: {'max_depth': 7, 'n_estimators': 88, 'min_samples_leaf': 6}. Best is trial 0 with value: 1.9489668821607211 # 6등
# Trial 1 finished with value: 1.9421799641969546 and parameters: {'max_depth': 11, 'n_estimators': 94, 'min_samples_leaf': 4}. Best is trial 1 with value: 1.9421799641969546.# 4등
# Trial 9 finished with value: 1.9404531955238757 and parameters: {'max_depth': 12, 'n_estimators': 88, 'min_samples_leaf': 7}. Best is trial 9 with value: 1.9404531955238757.
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# 상관계수 낮은 독립변수 5개 제거하니 조금더 나은 성능을 보임 0.0002 정도
# Trial 11 finished with value: 1.9402275876442183 and parameters: {'max_depth': 12, 'n_estimators': 98, 'min_samples_leaf': 6}. Best is trial 11 with value: 1.9402275876442183. # 4등